# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [37]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [38]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [39]:
import os
from glob import glob
import pandas as pd
# Write your code below.
price_data_dir = os.environ.get('PRICE_DATA')

parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive=True)     # Find the parquet files. Recursive search will be safer if paritioned in subfolders.


In [40]:
dd_px2 = dd.read_parquet(parquet_files).set_index("ticker")
dd_px2

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
ACH,datetime64[ns],float64,float64,float64,float64,float64,int64,string,int32
ACN,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [41]:
dd_feat = (
    dd_px2
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: (
                x.sort_values('Date', ascending=True)
                 .assign(
                     Close_lag_1=x['Close'].shift(1),
                     Adj_Close_lag_1=x['Adj Close'].shift(1)
                 )
            ),
            meta=pd.DataFrame(
                data={
                    'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8',
                },
                index=pd.Index([], dtype='string', name='ticker')
            )
        )
)


In [42]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1
npartitions=90,,,,,,,,,,,
ACH,object,object,object,object,object,object,object,object,object,object,object
ACN,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...


In [43]:


dd_feat = dd_feat.assign(
    returns=lambda x: x['Close'] / x['Close_lag_1'] - 1,
    hi_lo_range=lambda x: x['High'] - x['Low']
)
dd_feat





,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,
ACH,object,object,object,object,object,object,object,object,object,object,object,object,object
ACN,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


In [48]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACH,2001-12-11,4.6000,4.6500,4.4875,4.5675,3.760683,15192400.0,ACH.csv,2001,NaN,NaN,NaN,0.1625
ACH,2001-12-12,4.4875,4.4975,4.4250,4.4575,3.670114,3605600.0,ACH.csv,2001,10.33,10.33,-0.568490,0.0725
ACH,2001-12-13,4.4000,4.4375,4.3550,4.3625,3.591895,1921600.0,ACH.csv,2001,10.53,10.53,-0.585707,0.0825
ACH,2001-12-14,4.2875,4.2875,4.2500,4.2750,3.519851,1186000.0,ACH.csv,2001,10.78,10.78,-0.603432,0.0375
ACH,2001-12-17,4.3875,4.4500,4.3625,4.4500,3.663938,660400.0,ACH.csv,2001,10.57,10.57,-0.578997,0.0875
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.0600,4.5300,3.8800,4.5100,4.510000,1668500.0,ZIXI.csv,2020,8.66,8.66,-0.479215,0.6500
ZIXI,2020-03-27,4.4900,4.7100,4.1000,4.6000,4.600000,1146800.0,ZIXI.csv,2020,8.03,8.03,-0.427148,0.6100
ZIXI,2020-03-30,4.8300,4.8700,4.4400,4.6400,4.640000,1212000.0,ZIXI.csv,2020,7.65,7.65,-0.393464,0.4300


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
pdf = dd_feat.compute()

pdf = pdf.sort_values(['ticker', 'Date'])
pdf['returns_ma_10'] = (
    pdf.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)
)



In [51]:
pdf

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10
ticker,,,,,,,,,,,,,,
ACH,2001-12-11,4.6000,4.6500,4.4875,4.5675,3.760683,15192400.0,ACH.csv,2001,NaN,NaN,NaN,0.1625,NaN
ACH,2001-12-12,4.4875,4.4975,4.4250,4.4575,3.670114,3605600.0,ACH.csv,2001,4.5675,3.760683,-0.024083,0.0725,NaN
ACH,2001-12-13,4.4000,4.4375,4.3550,4.3625,3.591895,1921600.0,ACH.csv,2001,4.4575,3.670114,-0.021312,0.0825,NaN
ACH,2001-12-14,4.2875,4.2875,4.2500,4.2750,3.519851,1186000.0,ACH.csv,2001,4.3625,3.591895,-0.020057,0.0375,NaN
ACH,2001-12-17,4.3875,4.4500,4.3625,4.4500,3.663938,660400.0,ACH.csv,2001,4.2750,3.519851,0.040936,0.0875,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.0600,4.5300,3.8800,4.5100,4.510000,1668500.0,ZIXI.csv,2020,4.0000,4.000000,0.127500,0.6500,-0.000807
ZIXI,2020-03-27,4.4900,4.7100,4.1000,4.6000,4.600000,1146800.0,ZIXI.csv,2020,4.5100,4.510000,0.019956,0.6100,0.004262
ZIXI,2020-03-30,4.8300,4.8700,4.4400,4.6400,4.640000,1212000.0,ZIXI.csv,2020,4.6000,4.600000,0.008696,0.4300,0.015491


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? 
 No
+ Would it have been better to do it in Dask? Why?
For small Data converting to Pandas is better because it's faster to write, and easier to debug.
For Big Data converting to Pandas may crash the kernel which may end up in Memory Error. Dask handles large datasets by processing chunks lazily, allowing to compute features without loading everything into memory at once

(1 pt)


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.